# James' Preliminary Modeling Notebook

In [5]:
# import all relevant packages

import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout

%matplotlib inline

In [21]:
# define dataframes

train = pd.read_csv('./assets/train.csv')
test = pd.read_csv('./assets/test.csv')
weather = pd.read_csv('./assets/weather.csv')
spray = pd.read_csv('./assets/spray.csv')

In [3]:
train

Date                                            Address  \
0      2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1      2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2      2007-05-29  6200 North Mandell Avenue, Chicago, IL 60646, USA   
3      2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   
4      2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   
5      2007-05-29   1500 West Webster Avenue, Chicago, IL 60614, USA   
6      2007-05-29     2500 West Grand Avenue, Chicago, IL 60654, USA   
7      2007-05-29        1100 Roosevelt Road, Chicago, IL 60608, USA   
8      2007-05-29        1100 Roosevelt Road, Chicago, IL 60608, USA   
9      2007-05-29   1100 West Chicago Avenue, Chicago, IL 60642, USA   
10     2007-05-29    2100 North Stave Street, Chicago, IL 60647, USA   
11     2007-05-29    2200 North Cannon Drive, Chicago, IL 60614, USA   
12     2007-05-29    2200 North Cannon Drive, Chicago, IL 60614, USA   
13     2007-05-29     2200 West 113th Street, Chicago, IL 60643, USA   
14     2007-05-29     2200 West 113th Street, Chicago, IL 60643, USA   
15     2007-05-29   1100 South Peoria Street, Chicago, IL 60608, USA   
16     2007-05-29      1700 West 95th Street, Chicago, IL 60643, USA   
17     2007-05-29      2200 West 89th Street, Chicago, IL 60643, USA   
18     2007-05-29      2200 West 89th Street, Chicago, IL 60643, USA   
19     2007-05-29       North Streeter Drive, Chicago, IL 60611, USA   
20     2007-05-29       North Streeter Drive, Chicago, IL 60611, USA   
21     2007-05-29  6500 North Oak Park Avenue, Chicago, IL 60631,...   
22     2007-05-29   7500 North Oakley Avenue, Chicago, IL 60645, USA   
23     2007-05-29     1500 North Long Avenue, Chicago, IL 60651, USA   
24     2007-05-29  8900 South Carpenter Street, Chicago, IL 60620...   
25     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
26     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
27     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
28     2007-06-05    7900 West Foster Avenue, Chicago, IL 60656, USA   
29     2007-06-05    7900 West Foster Avenue, Chicago, IL 60656, USA   
...           ...                                                ...   
10476  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10477  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10478  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10479  2013-09-26    5800 North Pulaski Road, Chicago, IL 60646, USA   
10480  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10481  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10482  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10483  2013-09-26    9100 West Higgins Road, Rosemont, IL 60018, USA   
10484  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10485  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10486  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10487  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10488  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10489  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10490  2013-09-26   4800 West Montana Street, Chicago, IL 60639, USA   
10491  2013-09-26  5100 North Mont Clare Avenue, Chicago, IL 6065...   
10492  2013-09-26  5100 North Mont Clare Avenue, Chicago, IL 6065...   
10493  2013-09-26  8200 South Kostner Avenue, Chicago, IL 60652, USA   
10494  2013-09-26                  East 91st Place, Chicago, IL, USA   
10495  2013-09-26                  East 91st Place, Chicago, IL, USA   
10496  2013-09-26   1700 West Addison Street, Chicago, IL 60613, USA   
10497  2013-09-26          West Garfield Boulevard, Chicago, IL, USA   
10498  2013-09-26  1300 North Laramie Avenue, Chicago, IL 60651, USA   
10499  2013-09-26  1300 North Laramie Avenue, Chicago

In [22]:
test

Id        Date                                            Address  \
0            1  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1            2  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2            3  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
3            4  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
4            5  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
5            6  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
6            7  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
7            8  2008-06-11  4100 North Oak Park Avenue, Chicago, IL 60634,...   
8            9  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
9           10  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
10          11  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
11          12  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
12          13  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
13          14  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
14          15  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
15          16  2008-06-11  6200 North Mandell Avenue, Chicago, IL 60646, USA   
16          17  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
17          18  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
18          19  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
19          20  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
20          21  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
21          22  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
22          23  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
23          24  2008-06-11    7900 West Foster Avenue, Chicago, IL 60656, USA   
24          25  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
25          26  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
26          27  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
27          28  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
28          29  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
29          30  2008-06-11   1500 West Webster Avenue, Chicago, IL 60614, USA   
...        ...         ...                                                ...   
116263  116264  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116264  116265  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116265  116266  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116266  116267  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116267  116268  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116268  116269  2014-10-02      3400 West 77th Street, Chicago, IL 60652, USA   
116269  116270  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116270  116271  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116271  116272  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116272  116273  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116273  116274  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116274  116275  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116275  116276  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116276  116277  2014-10-02       5100 West 63rd Place, Chicago, IL 60638, USA   
116277  116278  2014-10-02  9600 South Longwood Drive, Chicago, IL 60643, USA   
116278  116279  2014-10-02  9600 South Longwood Drive, Chicago, IL 60643, USA   
116279  116280  2014-10-02  9600 South Longwood Drive, Chicago, IL 60643, USA   
116280  116281  2014-10-02  9600 South Long

In [23]:
weather

Station        Date  Tmax  Tmin Tavg Depart  DewPoint WetBulb Heat Cool  \
0           1  2007-05-01    83    50   67     14        51      56    0    2   
1           2  2007-05-01    84    52   68      M        51      57    0    3   
2           1  2007-05-02    59    42   51     -3        42      47   14    0   
3           2  2007-05-02    60    43   52      M        42      47   13    0   
4           1  2007-05-03    66    46   56      2        40      48    9    0   
5           2  2007-05-03    67    48   58      M        40      50    7    0   
6           1  2007-05-04    66    49   58      4        41      50    7    0   
7           2  2007-05-04    78    51    M      M        42      50    M    M   
8           1  2007-05-05    66    53   60      5        38      49    5    0   
9           2  2007-05-05    66    54   60      M        39      50    5    0   
10          1  2007-05-06    68    49   59      4        30      46    6    0   
11          2  2007-05-06    68    52   60      M        30      46    5    0   
12          1  2007-05-07    83    47   65     10        41      54    0    0   
13          2  2007-05-07    84    50   67      M        39      53    0    2   
14          1  2007-05-08    82    54   68     12        58      62    0    3   
15          2  2007-05-08    80    60   70      M        57      63    0    5   
16          1  2007-05-09    77    61   69     13        59      63    0    4   
17          2  2007-05-09    76    63   70      M        60      63    0    5   
18          1  2007-05-10    84    56   70     14        52      60    0    5   
19          2  2007-05-10    83    59   71      M        52      61    0    6   
20          1  2007-05-11    70    51   61      4        42      51    4    0   
21          2  2007-05-11    73    49   61      M        44      51    4    0   
22          1  2007-05-12    64    46   55     -2        36      46   10    0   
23          2  2007-05-12    65    47   56      M        37      46    9    0   
24          1  2007-05-13    69    43   56     -2        33      46    9    0   
25          2  2007-05-13    69    44   57      M        32      46    8    0   
26          1  2007-05-14    90    56   73     15        47      59    0    8   
27          2  2007-05-14    90    54   72      M        45      58    0    7   
28          1  2007-05-15    80    57   69     11        56      61    0    4   
29          2  2007-05-15    82    56   69      M        56      61    0    4   
...       ...         ...   ...   ...  ...    ...       ...     ...  ...  ...   
2914        1  2014-10-17    62    48   55      3        46      50   10    0   
2915        2  2014-10-17    63    49   56      M        48      52    9    0   
2916        1  2014-10-18    50    36   43     -8        38      42   22    0   
2917        2  2014-10-18    52    39   46      M        40      44   19    0   
2918        1  2014-10-19    54    33   44     -7        38      43   21    0   
2919        2  2014-10-19    57    36   47      M        40      44   18    0   
2920        1  2014-10-20    65    46   56      6        44      49    9    0   
2921        2  2014-10-20    66    50   58      M        46      51    7    0   
2922        1  2014-10-21    53    45   49     -1        39      44   16    0   
2923        2  2014-10-21    54    49   52      M        41      46   13    0   
2924        1  2014-10-22    56    35   46     -4        38      43   19    0   
2925        2  2014-10-22    56    43   50      M        38      44   15    0   
2926        1  2014-10-23    59    36   48     -1        40      45   17    0   
2927        2  2014-10-23    62    41   52      M        41      47   13    0   
2928        1  2014-10-24    67    52   60     11        46      52    5    0   
2929        2  2014-10-24    68    54   61      M        48      53    4    0   
2930        1  2014-10-25    72    49   61     13        40      51    4    0   
2931        2  2014-10-25    75    52

In [26]:
spray

Date        Time   Latitude  Longitude
0      2011-08-29  6:56:58 PM  42.391623 -88.089163
1      2011-08-29  6:57:08 PM  42.391348 -88.089163
2      2011-08-29  6:57:18 PM  42.391022 -88.089157
3      2011-08-29  6:57:28 PM  42.390637 -88.089158
4      2011-08-29  6:57:38 PM  42.390410 -88.088858
5      2011-08-29  6:57:48 PM  42.390395 -88.088315
6      2011-08-29  6:57:58 PM  42.390673 -88.088002
7      2011-08-29  6:58:08 PM  42.391027 -88.088002
8      2011-08-29  6:58:18 PM  42.391403 -88.088003
9      2011-08-29  6:58:28 PM  42.391718 -88.087995
10     2011-08-29  6:58:38 PM  42.392038 -88.087988
11     2011-08-29  6:58:48 PM  42.392410 -88.088078
12     2011-08-29  6:58:58 PM  42.392800 -88.088332
13     2011-08-29  6:59:08 PM  42.393257 -88.088370
14     2011-08-29  6:59:18 PM  42.393600 -88.088232
15     2011-08-29  7:00:08 PM  42.393203 -88.088417
16     2011-08-29  7:00:18 PM  42.392792 -88.088312
17     2011-08-29  7:00:28 PM  42.392477 -88.088210
18     2011-08-29  7:00:38 PM  42.392385 -88.088697
19     2011-08-29  7:00:48 PM  42.392098 -88.089113
20     2011-08-29  7:00:58 PM  42.392180 -88.089498
21     2011-08-29  7:01:08 PM  42.392428 -88.089790
22     2011-08-29  7:01:18 PM  42.392722 -88.090042
23     2011-08-29  7:01:28 PM  42.392958 -88.090300
24     2011-08-29  7:01:38 PM  42.392990 -88.090895
25     2011-08-29  7:01:48 PM  42.393002 -88.091612
26     2011-08-29  7:01:58 PM  42.393008 -88.092010
27     2011-08-29  7:02:08 PM  42.393020 -88.092548
28     2011-08-29  7:02:18 PM  42.393103 -88.093182
29     2011-08-29  7:02:28 PM  42.393118 -88.093737
...           ...         ...        ...        ...
14805  2013-09-05  8:28:41 PM  42.009215 -87.815097
14806  2013-09-05  8:28:51 PM  42.008713 -87.815108
14807  2013-09-05  8:29:01 PM  42.008163 -87.815073
14808  2013-09-05  8:29:11 PM  42.007722 -87.815087
14809  2013-09-05  8:29:21 PM  42.007200 -87.815078
14810  2013-09-05  8:29:31 PM  42.006698 -87.815073
14811  2013-09-05  8:29:41 PM  42.006262 -87.815047
14812  2013-09-05  8:29:51 PM  42.006235 -87.814400
14813  2013-09-05  8:30:01 PM  42.006277 -87.813815
14814  2013-09-05  8:30:11 PM  42.006818 -87.813775
14815  2013-09-05  8:30:21 PM  42.007355 -87.813768
14816  2013-09-05  8:30:31 PM  42.007818 -87.813775
14817  2013-09-05  8:30:41 PM  42.008387 -87.813745
14818  2013-09-05  8:30:51 PM  42.008933 -87.813720
14819  2013-09-05  8:31:01 PM  42.009487 -87.813683
14820  2013-09-05  8:31:11 PM  42.010087 -87.813695
14821  2013-09-05  8:31:21 PM  42.010682 -87.813713
14822  2013-09-05  8:31:31 PM  42.011213 -87.813688
14823  2013-09-05  8:31:51 PM  42.011340 -87.812347
14824  2013-09-05  8:33:11 PM  42.010135 -87.812330
14825  2013-09-05  8:33:21 PM  42.009693 -87.812338
14826  2013-09-05  8:33:31 PM  42.009125 -87.812338
14827  2013-09-05  8:33:41 PM  42.008507 -87.812357
14828  2013-09-05  8:33:51 PM  42.007897 -87.812335
14829  2013-09-05  8:34:01 PM  42.007218 -87.812338
14830  2013-09-05  8:34:11 PM  42.006587 -87.812355
14831  2013-09-05  8:35:01 PM  42.006192 -87.816015
14832  2013-09-05  8:35:21 PM  42.006022 -87.817392
14833  2013-09-05  8:35:31 PM  42.005453 -87.817423
14834  2013-09-05  8:35:41 PM  42.004805 -87.817460

[14835 rows x 4 columns]

In [ ]:
# courtesy of Brian Collins - copied

def date_separate(df):
    df = df.copy()
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day'] = pd.DatetimeIndex(df['Date']).day
    return df

# make all the dataset applied to the function
train=date_separate(train)
weather=date_separate(weather)
spray=date_separate(spray)

In [4]:
pandas_profiling.ProfileReport(train)

In [53]:
print("Train set data types:\n")
print(train.dtypes)
print("-----")
print("Weather set data types:\n")
print(weather.dtypes)
print("-----")
print("Spray set data types:\n")
print(spray.dtypes)

Train set data types:

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object
-----
Weather set data types:

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object
----

In [5]:
train.describe()

Block      Latitude     Longitude  AddressAccuracy  \
count  10506.000000  10506.000000  10506.000000     10506.000000   
mean      35.687797     41.841139    -87.699908         7.819532   
std       24.339468      0.112742      0.096514         1.452921   
min       10.000000     41.644612    -87.930995         3.000000   
25%       12.000000     41.732984    -87.760070         8.000000   
50%       33.000000     41.846283    -87.694991         8.000000   
75%       52.000000     41.954690    -87.627796         9.000000   
max       98.000000     42.017430    -87.531635         9.000000   

       NumMosquitos    WnvPresent  
count  10506.000000  10506.000000  
mean      12.853512      0.052446  
std       16.133816      0.222936  
min        1.000000      0.000000  
25%        2.000000      0.000000  
50%        5.000000      0.000000  
75%       17.000000      0.000000  
max       50.000000      1.000000

In [59]:
train['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [25]:
train_weather = train.merge(weather[weather['Station']==2], how='left', on='Date')

In [27]:
train_weather

Date                                            Address  \
0      2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
1      2007-05-29  4100 North Oak Park Avenue, Chicago, IL 60634,...   
2      2007-05-29  6200 North Mandell Avenue, Chicago, IL 60646, USA   
3      2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   
4      2007-05-29    7900 West Foster Avenue, Chicago, IL 60656, USA   
5      2007-05-29   1500 West Webster Avenue, Chicago, IL 60614, USA   
6      2007-05-29     2500 West Grand Avenue, Chicago, IL 60654, USA   
7      2007-05-29        1100 Roosevelt Road, Chicago, IL 60608, USA   
8      2007-05-29        1100 Roosevelt Road, Chicago, IL 60608, USA   
9      2007-05-29   1100 West Chicago Avenue, Chicago, IL 60642, USA   
10     2007-05-29    2100 North Stave Street, Chicago, IL 60647, USA   
11     2007-05-29    2200 North Cannon Drive, Chicago, IL 60614, USA   
12     2007-05-29    2200 North Cannon Drive, Chicago, IL 60614, USA   
13     2007-05-29     2200 West 113th Street, Chicago, IL 60643, USA   
14     2007-05-29     2200 West 113th Street, Chicago, IL 60643, USA   
15     2007-05-29   1100 South Peoria Street, Chicago, IL 60608, USA   
16     2007-05-29      1700 West 95th Street, Chicago, IL 60643, USA   
17     2007-05-29      2200 West 89th Street, Chicago, IL 60643, USA   
18     2007-05-29      2200 West 89th Street, Chicago, IL 60643, USA   
19     2007-05-29       North Streeter Drive, Chicago, IL 60611, USA   
20     2007-05-29       North Streeter Drive, Chicago, IL 60611, USA   
21     2007-05-29  6500 North Oak Park Avenue, Chicago, IL 60631,...   
22     2007-05-29   7500 North Oakley Avenue, Chicago, IL 60645, USA   
23     2007-05-29     1500 North Long Avenue, Chicago, IL 60651, USA   
24     2007-05-29  8900 South Carpenter Street, Chicago, IL 60620...   
25     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
26     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
27     2007-06-05  4100 North Oak Park Avenue, Chicago, IL 60634,...   
28     2007-06-05    7900 West Foster Avenue, Chicago, IL 60656, USA   
29     2007-06-05    7900 West Foster Avenue, Chicago, IL 60656, USA   
...           ...                                                ...   
10476  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10477  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10478  2013-09-26       South Cottage Grove Avenue, Chicago, IL, USA   
10479  2013-09-26    5800 North Pulaski Road, Chicago, IL 60646, USA   
10480  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10481  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10482  2013-09-26     4000 East 130th Street, Chicago, IL 60633, USA   
10483  2013-09-26    9100 West Higgins Road, Rosemont, IL 60018, USA   
10484  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10485  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10486  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10487  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10488  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10489  2013-09-26  ORD Terminal 5, O'Hare International Airport, ...   
10490  2013-09-26   4800 West Montana Street, Chicago, IL 60639, USA   
10491  2013-09-26  5100 North Mont Clare Avenue, Chicago, IL 6065...   
10492  2013-09-26  5100 North Mont Clare Avenue, Chicago, IL 6065...   
10493  2013-09-26  8200 South Kostner Avenue, Chicago, IL 60652, USA   
10494  2013-09-26                  East 91st Place, Chicago, IL, USA   
10495  2013-09-26                  East 91st Place, Chicago, IL, USA   
10496  2013-09-26   1700 West Addison Street, Chicago, IL 60613, USA   
10497  2013-09-26          West Garfield Boulevard, Chicago, IL, USA   
10498  2013-09-26  1300 North Laramie Avenue, Chicago, IL 60651, USA   
10499  2013-09-26  1300 North Laramie Avenue, Chicago

In [52]:
train_weather.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
Station                   0
Tmax                      0
Tmin                      0
Tavg                      0
Depart                    0
DewPoint                  0
WetBulb                   0
Heat                      0
Cool                      0
Sunrise                   0
Sunset                    0
CodeSum                   0
Depth                     0
Water1                    0
SnowFall                  0
PrecipTotal               0
StnPressure               0
SeaLevel                  0
ResultSpeed               0
ResultDir                 0
AvgSpeed                  0
dtype: int64

In [31]:
train_weather.describe()

Block      Latitude     Longitude  AddressAccuracy  \
count  10506.000000  10506.000000  10506.000000     10506.000000   
mean      35.687797     41.841139    -87.699908         7.819532   
std       24.339468      0.112742      0.096514         1.452921   
min       10.000000     41.644612    -87.930995         3.000000   
25%       12.000000     41.732984    -87.760070         8.000000   
50%       33.000000     41.846283    -87.694991         8.000000   
75%       52.000000     41.954690    -87.627796         9.000000   
max       98.000000     42.017430    -87.531635         9.000000   

       NumMosquitos    WnvPresent  Station          Tmax          Tmin  \
count  10506.000000  10506.000000  10506.0  10506.000000  10506.000000   
mean      12.853512      0.052446      2.0     81.986198     64.578336   
std       16.133816      0.222936      0.0      8.301924      7.393863   
min        1.000000      0.000000      2.0     58.000000     44.000000   
25%        2.000000      0.000000      2.0     78.000000     60.000000   
50%        5.000000      0.000000      2.0     83.000000     67.000000   
75%       17.000000      0.000000      2.0     87.000000     70.000000   
max       50.000000      1.000000      2.0     97.000000     79.000000   

          DewPoint   ResultSpeed     ResultDir  
count  10506.00000  10506.000000  10506.000000  
mean      59.58814      5.905854     17.758709  
std        7.87718      2.925563      9.041969  
min       38.00000      1.100000      1.000000  
25%       54.00000      4.100000     10.000000  
50%       60.00000      5.500000     19.000000  
75%       66.00000      7.800000     24.000000  
max       73.00000     15.400000     36.000000

In [32]:
train_weather.shape

(10506, 33)

In [33]:
train_weather.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg',
       'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset',
       'CodeSum', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure',
       'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [42]:
train_weather_carry = pd.get_dummies(train_weather['Species'])[['CULEX PIPIENS/RESTUANS','CULEX PIPIENS','CULEX RESTUANS']]

In [56]:
y_Train = train_weather.WnvPresent
 
X_Train = pd.DataFrame()
X_Test = pd.DataFrame()

In [49]:
X_train['Latitude'] = train_weather.Latitude
X_train['Tmin'] = train_weather['Tmin']
X_train['Longitude'] = train_weather.Longitude
X_train['Tmax'] = train_weather.Tmax
X_train['AvgSpeed'] = train_weather['AvgSpeed']
X_train['DewPoint'] = train_weather.DewPoint
X_train['DayOfYear'] = pd.to_datetime(train_weather['Date'])
X_train['CULEX PIPIENS'] = train_weather_carry['CULEX PIPIENS']
X_train['CULEX RESTUANS'] = train_weather_carry['CULEX RESTUANS']
X_train['WetBulb'] = train_weather.WetBulb

X_train.shape

(10506, 10)

In [50]:
X_Train.dtypes

Latitude                 float64
Tmin                       int64
Longitude                float64
Tmax                       int64
DewPoint                   int64
DayOfYear         datetime64[ns]
CULEX PIPIENS              uint8
CULEX RESTUANS             uint8
WetBulb                   object
AvgSpeed                  object
dtype: object

In [61]:
model = GradientBoostingClassifier(max_features = 6, max_depth = 100) 
scores = cross_val_score(model, X_Train, y_Train, cv=3)
print(scores)
print(np.mean(scores))

ValueError: Found input variables with inconsistent numbers of samples: [0, 10506]

In [58]:
model = RandomForestClassifier(max_features = 6, max_depth = 20) 
scores = cross_val_score(model, X_Train, y_Train, cv=3)
print(scores)
print(np.mean(scores))

ValueError: Found input variables with inconsistent numbers of samples: [0, 10506]